In [14]:
!pip install requests
!pip install bs4
!pip install lxml
!pip install pandas

     ---------------------------------------- 0.0/61.2 kB ? eta -:--:--
     ------ --------------------------------- 10.2/61.2 kB ? eta -:--:--
     ------------------------- ------------ 41.0/61.2 kB 487.6 kB/s eta 0:00:01
     -------------------------------------- 61.2/61.2 kB 542.2 kB/s eta 0:00:00
     ---------------------------------------- 0.0/341.8 kB ? eta -:--:--
     ---------- ---------------------------- 92.2/341.8 kB 2.6 MB/s eta 0:00:01
     ------------------- ------------------ 174.1/341.8 kB 2.1 MB/s eta 0:00:01
     -------------------------------------  337.9/341.8 kB 2.6 MB/s eta 0:00:01
     -------------------------------------- 341.8/341.8 kB 2.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   - -------------------------------------- 0.4/10.6 MB 13.5 MB/s eta 0:00:01
   --- ------------------------------------ 0.8/10.6 MB 8.9 MB/s eta 0:00:02
   --- ------------------------------------ 0.9/10.6 MB 8.5 MB/s eta 0:00:02


In [15]:
# import libraries
import os
import pandas as pd
import lxml
import requests  
from bs4 import BeautifulSoup

In [16]:
def read_txt_file(file_path):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
            return lines
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

match_url_list = read_txt_file(os.getcwd() + '\data\Ascension.txt')
match_url_list

['https://www.vlr.gg/237652/apeks-vs-gentle-mates-champions-tour-2023-emea-ascension-gf\n',
 'https://www.vlr.gg/237655/acend-vs-gentle-mates-champions-tour-2023-emea-ascension-lbf\n',
 'https://www.vlr.gg/237654/gentle-mates-vs-saw-champions-tour-2023-emea-ascension-lr1\n',
 'https://www.vlr.gg/237649/acend-vs-gentle-mates-champions-tour-2023-emea-ascension-ubsf\n',
 'https://www.vlr.gg/235029/gentle-mates-vs-apeks-champions-tour-2023-emea-ascension-d4\n',
 'https://www.vlr.gg/235023/gentle-mates-vs-fokus-champions-tour-2023-emea-ascension-d3\n',
 'https://www.vlr.gg/235018/gentle-mates-vs-case-esports-champions-tour-2023-emea-ascension-d2\n',
 'https://www.vlr.gg/235016/team-falcons-vs-gentle-mates-champions-tour-2023-emea-ascension-d1']

In [19]:
url = 'https://www.vlr.gg/235016/team-falcons-vs-gentle-mates-champions-tour-2023-emea-ascension-d1'
source_match = requests.get(url=url).text

In [20]:
source_match

'<!DOCTYPE html>\n<html lang="en"> \n\t<head>\n\t\t<title>\n\t\t\tTeam Falcons vs. Gentle Mates | Champions Tour 2023 EMEA: Ascension | Group Stage | Valorant match | VLR.gg\t\t</title>\n\n\t\t\t\t\t<meta name="description" content="Stats, score, streams, and VODs from Team Falcons vs. Gentle Mates - Day 1 match of Champions Tour 2023 EMEA: Ascension Valorant event">\n\t\t\t\t<meta charset="utf-8">\n\n\t\t<meta property="og:title" content="Team Falcons vs. Gentle Mates | Champions Tour 2023 EMEA: Ascension | Group Stage | Valorant match | VLR.gg">\n\t\t<meta property="og:site_name" content="VLR.gg">\n\n\t\t\t\t\t<meta property="og:description" content="Stats, score, streams, and VODs from Team Falcons vs. Gentle Mates - Day 1 match of Champions Tour 2023 EMEA: Ascension Valorant event">\n\t\t\n\t\t\t\t\t\n\t\t\t\t\t\t\t<meta property="og:image" content="/img/vlr/favicon.png">\n\t\t\t\t\t\n\t\t\t\t\t<meta name="twitter:card" content="summary">\n\t\t\n\t\t\t\t\t<meta name="twitter:site" 

In [24]:
match_stats = []

for matchnum in range(len(match_url_list)):
    url = match_url_list[matchnum]
    
    source_match = requests.get(url=url).text
    soup_match = BeautifulSoup(source_match, features="html.parser")
    
    table_match = soup_match.findAll('table', {'class':'wf-table-inset mod-overview'})

    stage = soup_match.findAll('div', {'class':'match-header-event-series'})[0].text.strip().split(":", 1)[0]

    series = soup_match.findAll('div', {'class':'match-header-event-series'})[0].text.strip().split("\n", 1)[1].strip()
    
    for a in [a for a in list(range(len(table_match))) if a not in [2,3]]:
        if a in [0,1]:
            map_num = 1
        else:
            map_num = a//2
        
        map_name = soup_match.findAll('div', {'class':'vm-stats-gamesnav-item js-map-switch'})[map_num-1].text.strip()[1:].strip()
        stage = soup_match.findAll('div', {'class':'match-header-event-series'})[0].text.strip().split(":", 1)[0]
        team_name = soup_match.findAll('div', {'class':'wf-title-med'})[a % 2].text.strip()
        
        table = table_match[a]
        headers_match = []
        
        for i in table.find_all('th'):
            title_match = i.text.strip()
            headers_match.append(title_match)
            
        headers_match[0] = 'Player Name'
        headers_match[1] = 'Agent Name'
        headers_match[6] = 'K/D +/–'   
        headers_match[12] = 'FK/FD +/–'   
        df_match = pd.DataFrame(columns=headers_match)
        
        for row in table.find_all('tr')[1:]:
            data = row.find_all('td')
            row_data = [td.text.strip() for td in data]
            row_data[1] = str(data[1]).split('title="',1)[1].split('"',1)[0].title()
            length = len(df_match)
            df_match.loc[length] = row_data
        
        df_match['Team Name'] = team_name
        df_match['Map Name'] = map_name
        df_match['Map #'] = map_num
        df_match['Stage'] = stage
        df_match['Series'] = series
        
        match_stats.append(df_match)

In [36]:
match_stats

[                                         Player Name Agent Name  \
 0  MAGNUM \n\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\...    Killjoy   
 1  Enzo \n\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\...       Sova   
 2  ShadoW \n\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\...     Breach   
 3  Keiko \n\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t...       Jett   
 4  Mistic \n\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\...       Omen   
 
                   R            ACS          K                    D   K/D +/–  \
 0  0.98\n0.98\n0.98  225\n237\n206  15\n10\n5   /\n\n14\n9\n5\n\n/   1\n1\n0   
 1  0.94\n0.75\n1.26  191\n132\n293   10\n4\n6   /\n\n14\n8\n6\n\n/  10\n5\n5   
 2  0.85\n0.93\n0.73  165\n174\n152   12\n9\n3   /\n\n14\n8\n6\n\n/  11\n9\n2   
 3  0.84\n1.13\n0.35  214\n278\n107  14\n12\n2   /\n\n16\n9\n7\n\n/   3\n2\n1   
 4  0.70\n0.53\n0.98  130\n117\n155    8\n4\n4  /\n\n15\n10\n5\n\n/   4\n1\n3   
 
           +/–           KAST            ADR            HS%       FK FK/FD +/–  \
 0   +1\n+1\n0  74%\